# RA2CE feature: Losses analysis

In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import rasterio
from branca.colormap import LinearColormap
import folium

from ra2ce.ra2ce_handler import Ra2ceHandler #import the ra2cehandler to run ra2ce analyses

## Single link losses Analysis

In [3]:
root_dir = Path(".\\data\\single_link_losses")

network_ini = root_dir / "network.ini"
analysis_ini = root_dir / "analysis.ini"

handler = Ra2ceHandler(network=network_ini, analysis=analysis_ini)

handler.configure()
handler.run_analysis()

2024-03-26 08:12:26 AM - [ra2ce_logging.py:41] - root - INFO - RA2CE logger initialized.
2024-03-26 08:12:26 AM - [network_wrapper_factory.py:56] - root - INFO - Start creating a network from the submitted shapefile.
2024-03-26 08:12:26 AM - [vector_network_wrapper.py:120] - root - INFO - Read files data\single_link_losses\static\network\Red_Vial_RD_wgs84.shp into a 'GeoDataFrame'.
2024-03-26 08:12:26 AM - [vector_network_wrapper.py:92] - root - INFO - reproject vector file to project crs
2024-03-26 08:12:43 AM - [multi_graph_network_exporter.py:55] - root - INFO - Saved base_graph in C:\repos\ra2ce\examples\data\single_link_losses\static\output_graph.
2024-03-26 08:12:43 AM - [networks_utils.py:1222] - root - INFO - Saving nodes as shapefile: data\single_link_losses\static\output_graph\base_graph_nodes.gpkg
2024-03-26 08:12:43 AM - [networks_utils.py:1223] - root - INFO - Saving edges as shapefile: data\single_link_losses\static\output_graph\base_graph_edges.gpkg
2024-03-26 08:12:44 A

[AnalysisResultWrapper(analysis_result=                link_type                                           geometry  \
 ID                                                                             
 1050           Regional-S  LINESTRING Z (-71.19405 18.24858 0.00000, -71....   
 934            Regional-S  LINESTRING Z (-71.08824 18.08410 0.00000, -71....   
 933            Regional-S  LINESTRING Z (-71.08319 18.09459 0.00000, -71....   
 790   Caminos Vecinales-S  LINESTRING Z (-71.06260 18.32464 0.00000, -71....   
 785   Caminos Vecinales-S  LINESTRING Z (-71.65068 17.97898 0.00000, -71....   
 ...                   ...                                                ...   
 726            Regional-S  LINESTRING Z (-71.20752 18.24914 0.00000, -71....   
 727            Regional-S  LINESTRING Z (-71.20965 18.24900 0.00000, -71....   
 756               Local-S  LINESTRING Z (-71.16866 18.38792 0.00000, -71....   
 759               Local-S  LINESTRING Z (-71.18044 18.38464 0.00000, -

## Multi-link losses Analysis

In [ ]:
root_dir_multi = Path(".\\data\\multi_link_losses")

network_ini_multi = root_dir_multi / "network.ini"
analysis_ini_multi = root_dir_multi / "analysis.ini"

handler_multi = Ra2ceHandler(network=network_ini_multi, analysis=analysis_ini_multi)

handler_multi.configure()
handler_multi.run_analysis()